In [ ]:
# MinIO connection details
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = os.getenv('AWS_S3_BUCKET')
region = os.getenv('AWS_DEFAULT_REGION')

# Initialize the S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

In [ ]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, x_train.shape[1]]))]

# Convert the model to ONNX
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
with open("linear_regression.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

print("Model has been converted and saved as linear_regression.onnx")

In [ ]:
onnx_model_file = 'linear_regression.onnx'
models_bucket = 'models'
# Upload the ONNX model file to MinIO bucket
with open(onnx_model_file, "rb") as f:
    s3_client.upload_fileobj(f, models_bucket, 'test/regression/1/' + onnx_model_file)

print(f"ONNX model '{onnx_model_file}' uploaded to bucket '{bucket_name}' on MinIO.")